# NAVER TRENDS CRAWLING

In [ ]:
%run ~/MovieTrends/src/11_NT_define.ipynb

## Execution

#### Load Data

In [ ]:
api_info = {
    "client_id": "fe5kOiBVrMopPhGmaeeh",
    "client_secret": "zauzLV_BPn",
    "url": "https://openapi.naver.com/v1/datalab/search"
}

In [ ]:
api_info = {
    "client_id": "6GPEYsepjIil0OCSMnU5",
    "client_secret": "QUd5qt5QU9",
    "url": "https://openapi.naver.com/v1/datalab/search"
}

In [ ]:
# mv2019 = pd.read_table("~/MovieTrends/data/movie_data_200610.csv", sep = ",", encoding = "UTF-8", header=0, names = ["Title", "kwd1", "kwd2", "kwd3", "OpenDate"])
# mv2019.OpenDate = pd.Series([pd.Timestamp(datetime.strptime(i, '%Y%m%d')) for i in map(str, mv2019.OpenDate)])
# mv2019["Movie_kwd"] = [json.dumps([itm for itm in [row.Title, row.kwd1, row.kwd2, row.kwd3] if itm is not np.nan], ensure_ascii = False) for idx, row in mv2019.iterrows()]

In [ ]:
mv_std_info = pd.read_table("~/MovieTrends/data/mv_std_info.dat", sep = "|", header=0, encoding = "UTF-8", names = ['mv_id', 'title', 'kwd1', 'kwd2', 'kwd3', 'kwdcb', 'open_date'])
mv_std_info['open_date'] = pd.Series([pd.Timestamp(datetime.strptime(i, '%Y%m%d')) for i in map(str, mv_std_info.open_date)])
mv_std_info['Movie_kwd'] = [json.dumps([itm for itm in [row.title, row.kwd1, row.kwd2, row.kwd3] if itm is not np.nan], ensure_ascii = False) for idx, row in mv_std_info.iterrows()]

#### Get Trend Index

In [ ]:
NT = NaverTrend()
NT.set_api_info(api_info)
NT.load_data(mv_std_info)

In [ ]:
NT.set_movie(max = True)

In [ ]:
target_subset_list = [[*NT.movie_list_full[idx:idx+4]] for idx in [*range(0, len(NT.movie_list_full), 4)]]
result_list = [NT.get_trend(target_subset, True) for target_subset in target_subset_list]

In [ ]:
result_full = pd.concat(result_list)
result_full

In [ ]:
result_merged = result_full.merge(NT.movie_data_full, how="left", left_on="Keyword", right_on="title").drop("title", 1)
result_merged["D"] = result_merged.Time - result_merged.open_date
result_merged.D = [i.days for i in result_merged.D]
result_merged = result_merged[['mv_id', 'Keyword', 'open_date', 'D', 'Time', 'TrendIndex']]
result_merged.columns = ['mv_id', 'movie_nm', 'open_date', 'D', 'date', 'trendindex'] 

In [ ]:
result_merged.head(10)

In [ ]:
result_merged.to_csv("~/MovieTrends/output/ts_trend_index.dat", sep="|", index=False)

#### Aggregate Time Index

In [ ]:
rng1, rng2, rng3 = [i for i in range(-30, 0)], [i for i in range(0, 30)], [i for i in range(30, 60)]
rng = rng1 + rng2 + rng3

result_merged['group_tf'] = pd.Series([_ in rng for _ in result_merged.D])
result_merged['ogroup'] = pd.Series([1 if _ in rng1 else 2 if _ in rng2 else 3 if _ in rng3 else 0 for _ in result_merged.D])

ti_aggr = result_merged[result_merged.group_tf == True].groupby(['mv_id', 'movie_nm', 'ogroup']).agg({'trendindex': ['mean', 'min', 'max']})
ti_aggr.columns = ['ti_mean', 'ti_min', 'ti_max']
ti_aggr = ti_aggr.reset_index()

In [ ]:
ti_aggr.head(10)

In [ ]:
ti_aggr.to_csv("~/MovieTrends/output/ts_trend_index_aggr.dat", sep="|", index=False)

---

### TEST

In [ ]:
body = NT._define_body(["데드풀 2: 순한맛"])
request = urllib.request.Request("https://openapi.naver.com/v1/datalab/search")
request.add_header("X-Naver-Client-Id", "6GPEYsepjIil0OCSMnU5")
request.add_header("X-Naver-Client-Secret", "QUd5qt5QU9")
request.add_header("Content-Type", "application/json")
response = urllib.request.urlopen(request, data = body.encode("utf-8"))

In [ ]:
NT._response_to_df(NT._get_response_body(NT._define_body(["데드풀 2: 순한맛"])))

In [ ]:
body = NT._define_body(["데드풀 2: 순한맛"])

In [ ]:
request = urllib.request.Request("https://openapi.naver.com/v1/datalab/search")
request.add_header("X-Naver-Client-Id", "6GPEYsepjIil0OCSMnU5")
request.add_header("X-Naver-Client-Secret", "QUd5qt5QU9")
request.add_header("Content-Type", "application/json")
response = urllib.request.urlopen(request, data = body.encode("utf-8"))

In [ ]:
NT.get_trend(["데드풀 2: 순한맛"])

In [ ]:
NT.movie_data_full[NT.movie_list_full == '그때 그들 확장판 1: 욕망의 부활']

In [ ]:
target_list = ['타짜: 원 아이드 잭', '백두산']
result = NT.get_trend(target_list)
result.iloc[result.TrendIndex.idxmax()]

In [ ]:
NT._define_body(target_list)

In [ ]:
result[result.Time == '2019-04-15']

In [ ]:
movie_list = ['타짜: 원 아이드 잭', '백두산']
movie_kwd_list = NT.movie_data_full[NT.movie_list_full.isin(movie_list)]["Movie_kwd"]
movie_list = NT.movie_data_full[NT.movie_list_full.isin(movie_list)]["Title"]
keywords_list = ["{\"groupName\": \"" + movie + "\", \"keywords\": " + str(movie_kwd) + "}" for movie, movie_kwd in zip(movie_list, movie_kwd_list)]
body ="{\"startDate\":\"2018-12-01\", \"endDate\":\"2020-02-28\", \"timeUnit\":\"date\", \"keywordGroups\":" + "[" + ", ".join(keywords_list) + "]}"

In [ ]:
body